In [52]:
##set up
import numpy as np
import pandas as pd
import string
import email
import os ##use this to traverse folders
from email.parser import Parser
import re
import lxml.html
import collections

In [53]:
#extract labels from TREC2007/full/ file lists
with open('trec07p/full/index', 'r') as ifile:
    raw_labels = ifile.readlines()

#using file list to give labels and e-mail numbers

categories = [i.split( )[0] for i in raw_labels]
email_nums = [i.split(".")[-1][:-1] for i in raw_labels]
labels = pd.Series(categories, index=email_nums)

In [131]:
#reference from other place

# parser = Parser()
# corpus = []
# for email_num in email_nums:
#     filepath = 'trec07p/data/inmail.%s' % email_num

#     with open(filepath, 'r',) as ifile:
#         email = parser.parse(ifile)

#     ## TODO also keep the subject header

#     ## code and details from:
#     ## https://stackoverflow.com/questions/17874360/python-how-to-parse-the-body-from-a-raw-email-given-that-raw-email-does-not/32840516#32840516
#     if email.is_multipart():
#         for part in email.walk():
#             content_type = part.get_content_type()
#             content_dispo = str(part.get('Content-Disposition'))

#             # skip any text/plain (txt) attachments
#             if content_type == 'text/plain' and 'attachment' not in content_dispo:
#                 body = part.get_payload(decode=False)
#                 break ## only keep the first email
#     else:
#         body = email.get_payload(decode=False)

#     ## TODO strip html stuff ? or valuable features ?
#     try:
#         corpus.append((email_num, body))
#     except NameError:
#         pass

# email_nums, corpus = zip(*corpus)

In [18]:
# extract the body of email
def walkMsg(msg):
    """
    Input: string of email
    Output: body of email
    """
    for part in msg.walk():
        if part.get_content_type() == "multipart/alternative":
            continue
        yield part.get_payload(decode=1)

In [19]:
##encode
def robust_decode(bs):
    '''Takes a byte string as param and convert it into a unicode one.
First tries UTF8, and fallback to Latin1 if it fails'''
    cr = None
    try:
        cr = bs.decode('utf8')
    except UnicodeDecodeError:
        cr = bs.decode('ISO-8859-1')
    return cr

In [46]:
def info_of_email_2():
    """
    Input: path, list of files name, classification(spam or ham)
    Ourput: a list with info of email
    """
    
    parser = Parser()
    info = []
    for nums in range(len(categories)):
        filepath = 'trec07p/data/inmail.%s' % email_nums[nums]
        
        with open(filepath, 'r') as ifile:
            email = parser.parse(ifile)
            
            Date = email.get("Date")
            Content = " ".join([robust_decode(i) for i in walkMsg(email) if i != None]).replace("\n"," ").replace("\t"," ").replace("r"," ")
            if 'html' in Content:
                Content = lxml.html.fromstring(Content).text_content()
            try:
                Date = re.split(':|,| ',Date)
                info.append([Date[4], Date[3], Date[2], Date[0], Date[5],
                             email.get("From"),email.get('To'),
                             email.get('cc'),email.get('bcc'), 
                             str(email.get("Content-Type")).split(";")[0],
                             email.get('subject'),
                             Content,
                             categories[nums]])
            except:
                info.append([[], [], [], [], [],
                             email.get("From"), email.get('To'),
                             email.get('cc'), email.get('bcc'), 
                             str(email.get("Content-Type")).split(";")[0],
                             email.get('subject'),
                             Content,
                             categories[nums]])
    return info

In [47]:
df = pd.DataFrame(info_of_email_2(),
                 columns=['Year','Month','Day','Weekday',"Hour",
                                'From','To','cc','bcc','Content_type','Subjects','Content','Classification'])

In [50]:
df.head()

,Year,Month,Day,Weekday,Hour,From,To,cc,bcc,Content_type,Subjects,Content,Classification
0,2007,Apr,08,Sun,21,"""Tomas Jacobs"" <RickyAmes@aol.com>",the00@speedy.uwaterloo.ca,None,None,multipart/alternative,"Generic Cialis, branded quality@",Do you feel the p essu e to pe fo m and...,spam
1,2007,Apr,08,Sun,12,Yan Morin <yan.morin@savoirfairelinux.com>,debian-mirrors@lists.debian.org,None,None,text/plain,Typo in /debian/README,"Hi, i've just updated f om the gulus and I che...",ham
2,2007,Apr,8,Sun,17,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...",<the00@plg.uwaterloo.ca>,None,None,multipart/alternative,authentic viagra,Mega authenticV I A G R A $ DISCOUNT p iceC...,spam
3,2007,Apr,08,Sun,17,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>",opt4@speedy.uwaterloo.ca,None,None,text/plain,Nice talking with ya,"Hey Billy, it was eally fun going out the ...",spam
4,2007,Apr,8,Sun,19,"""Christi T. Jernigan"" <dcube@totalink.net>",ktwarwic@speedy.uwaterloo.ca,None,None,multipart/related,or trembling; stomach cramps; trouble in sleep...,"system"" of the home. It will have the capabil...",spam


In [57]:
collections.Counter(int(df['Year']))

TypeError: cannot convert the series to <type 'int'>

In [ ]:
filepath = 'trec07p/data/inmail.%s' % email_nums[nums]

with open(filepath, 'r') as ifile:
    email = parser.parse(ifile)